# Spotify API

Retrive the songs and its metrics from the public playlits that Spotify has

In [ ]:
#@title Libraries
import requests
from google.colab import drive
import pandas as pd
import os
from bs4 import BeautifulSoup
import json
from collections import Counter
import numpy as np
import math

In [ ]:
#@title Tokens and ids
AUTH_TOKEN = 'BQDc7x7FHm9iIWvnKaz54cOew5NxSDxvATKvKC6JlcGXSe7wgIrAAoZHEUiXCPvxrwEBGO0cEl5D8TqHNzWq8ECBfVkCrEkwgbN2UDEtuMLBNEnBvMnB1i22qWMhZxg0SDTv5sDt0zMw-WFMXCWzE1hP0YODtzyUMC-eix29EoQ8wD8'
headers = {"Authorization": "Bearer {}".format(AUTH_TOKEN)}

id_almu = '1535d2051a1c4516b7cad44aec63097b'
id_ines = '67bf8f81591443f7bdcd562299c6e4c1'
id_pau = '7bdb591f49184e2d867c5827e7fc93dd'

In [ ]:
def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [ ]:
#@title Connecting to Drive

drive.mount('/content/drive')

Mounted at /content/drive


## Functions

1. retrive_songs(playlist_id) --> returns list of song id´s
2. parse_playlist(playlist)
3. get_audio_features(song_ids)
4. parse_audio_features(audio, playlist_dict, song_ids)
5. create_playlist_dict(playlist_id)
6. check_id_playlists(json_response, playlists_ids={})

In [ ]:
def retrieve_songs(playlist_id):
    song_ids = []
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks?fields=items(track(name,id,artists(name)))'
    response = requests.get(url,
        headers = {
            'Authorization': f'Bearer {AUTH_TOKEN}'
        }        
    )
    
    json_response = response.json()

    res = json_response['items']            
    for song in res:
      try:
        song_ids.append(song['track']['id'])
      except:
        pass

    return song_ids
    

In [ ]:
def parse_playlist(playlist):
    playlist_dict = {}
    print(playlist[0])
    for i in playlist:
        song = i['track']['name']
        id_ = i['track']['id']
        artists = i['track']['artists']
        
        #playlist_dict[id_] = {'name': song}
        playlist_dict[id_] = {'name': song, 'artists': artists}  # necesario para lyrics en genius
    return playlist_dict

In [ ]:
def get_audio_features(song_ids):
    song_ids_parsed = ','.join(song_ids)
    url = f'https://api.spotify.com/v1/audio-features?ids={song_ids_parsed}'
    response = requests.get(url,
        headers = {
            'Authorization': f'Bearer {AUTH_TOKEN}'
        }        
    )
    json_response = response.json()
    
    return json_response

In [ ]:
def parse_audio_features(audio, playlist_dict, song_ids):
    for i in range(len(playlist_dict)):
        playlist_dict[song_ids[i]]['audio_features'] = audio['audio_features'][i]        
    return playlist_dict

In [ ]:
def create_playlist_dict(playlist_id):
    playlist = parse_playlist(retrieve_songs(playlist_id))
    
    song_ids = []
    for song in playlist:
        song_ids.append(song)
    song_ids_str = str(song_ids)
    parsed_song_ids = song_ids_str.replace("'", "").replace('[', '').replace(']', '').replace(' ','')
    
    res = parse_audio_features(get_audio_features(parsed_song_ids), playlist, song_ids)
    
    return res

In [ ]:
def check_id_playlists(json_response, playlists_ids={}):
  for i in json_response['items']:
  #check the name with: Top 50 - (starts with)
  #sacar info de 'uri' _ 'spotify:playlist: id' ESTE ES EL ID QUE NECESITAMOS

    if i['name'].startswith('Top\xa050:'):
      n = i['name'].split('Top\xa050: ')[1]
      playlists_ids[n]=i['uri'].split(':')[-1]
    
  return playlists_ids

In [ ]:
def get_song_popularity(song_id_list):
  buckets = math.ceil(len(song_id_list)/50)
  songs = np.array_split(song_id_list, buckets)
  song_pop = list()
  for song_list in songs:
    song_list =  ','.join(song_list)
    url = f'https://api.spotify.com/v1/tracks?ids={song_list}'
    response = requests.get(url,
      headers = {
          'Authorization': f'Bearer {AUTH_TOKEN}'
      }        
    )
    json_response = response.json()
    for track in json_response['tracks']:
      song_pop.append(track['popularity'])

  return song_pop


# get_song_popularity(['0mzvDQW1IgLuXAwD5YMN6i', '1L4f4kyGeaLdZeVhPLaD8j', '4IKSehYf4e9tjijDSZHQF2', '5vK8ILrHjXjdZJHp0AtEVJ', '7tAsrC5tNcbccUmMnRsmZI', '25mYG9U2wb2UnQrOVkSIbE', '1R9dlH1uHHuJJF8gp9WN19', '3uPrQDkNYwbOBwX5SHIXb4', '06Uhw7KLSBqAJqYy2fdjiN', '3FiX402udtjX777EHZUAbU', '7ayopKtCQTOEDpRB5xpU6I', '6oNF4mTDylJfuQF9Wqb3Hw', '1Z0GfmDY4xTdRNOCGWckyV', '2SwDECpUAk8jtZ31aXYQTe', '0jniFtpAyWN1hCtQdTllSg', '7rvKogufniHwoxQIgdcGde', '06JbxvU7hdbBw1eoEOUWW6', '2W48o4UcesMLJZMwoL6B3o', '2fDaX8wgmwzSYmdLmTDAIw', '1eKhGmIxCVwXHjxrh3aqT5', '7nByHsiIV08TNL6hG4nBBs', '0vC73pf42tqNW2ecQrdNtm', '36KSajo0NyH3eGgbS4AuJ7', '40ooZvE9wvUQ8oNfSzs73N', '4RIi1gNmKDzSH04Vvws2DK', '5uOdakRKQbRu6HJnorQ5on', '4hMKS6gf79eVQ70AkAXyCE', '2qSmlhY4zy0sZaGeKowauQ', '2vMRmWkG5H3Bi93ufKmWNl', '59Gf13MS5B8IyLG0ZjpWFX', '0zd0kuNdqeLgxuFm5SLYxH', '2VA9AQAyYgItqq1KE3AtKr', '555uSzxhjD72TQBEoPOfYT', '4Wdin6LVBOduIVlCKi8FYK', '4VD1tBHGujKPhkaEji4v5Q', '3JvKfv6T31zO0ini8iNItO', '3TArda30KGDCgfB3U9fmrC', '6gpTColYGFZ1q0isxHoE0u', '7Kn7TYpJT0fjv5he98acyq', '1H3uUBhA5X9YNdk1Ciw8ky', '3qDRfkAQLT4yQPtzYh9nrA', '2tKSIeksPzMglcIJVc83ID', '3bc2L7xATgHEXtNJdEJOv9', '4wwz9Ft24R5WOgWpZSOIo5', '1mFh7ZsIQvy10e3hGPoXQA', '1JoZpWoOpDZWCbfhVgjRiC', '7siCyYmarG5TfLQbWlNKJ7', '1s8QRwM3pW5ekIUKwlHsxo', '6OdcsvgPkIFmiyqDPgCaDy', '4jEEloo2pedhkrP5NBTl85', '2cJCPpy5Iwgon0wORjxJan', '2odlHl0cZ7Q1npc6RL07Ar', '7ifjN5DxB3r1hKnqjRDO1a', '53y2hHY9gPhOZzoEh7umgq', '14d79QOV2pvJHH9vlUfZwJ', '4OkXQF0aJwj3eECnph73GC', '4OxSe2OdRDAEx8BO3CA5uD', '3Dvams9yiJchXGj1OlOCil', '66tgeHKe9FfKCQnrfkFNxy', '4qu63nuBpdn0qHUHuObEj1', '6hYidRx2EkQ39bmZHvqqaI', '5sNLvOP38aQxpjUeJ2VQsP', '0wzK2m89t22oOayHYfV75U', '7s61Kg0qZhhFj5MQW7DDIB', '1D3irR3juCW9kOHfW1LXws', '11BAVEGi1ivJ6JWLqKUNrZ', '6vRCKJDtnrJEcyyzIMa0w0', '0JOHC5pkFmkZSLkwiumjTt', '5b19IPegprOMgLPDtC1Cyf', '7vEQ7hcDYobr5p4OkejYGK', '7k0wVBW3cvV6Xmb87JikIf', '1rEHH6Wrm0iuv61O1oM2nc', '1HNkqx9Ahdgi1Ixy2xkKkL', '7795WJLVKJoAyVoOtCWqXN', '3Vjmj8370BnHk6b3viNxfU', '0v1g5U1OAb0XBjKyA8R36i', '5Ohxk2dO5COHF1krpoPigN', '75JFxkI2RXiU7L9VXzMkle', '4PdLup4OaCGktZG8cvbkBz', '0fD6vPYWty2Jy4VVozWzfp', '5RoHs6F9G5eH1kl488diEu', '09wrW4mdaoBrOFY6KSR94h', '2WtPwpBC8qCRtzHUDZgwBN', '4m4dmiW49WkRkSWliJ5Zoj', '6sbNTk4raoJtHh92BUbLoi', '7ce20yLkzuXXLUhzIDoZih', '7GXG38022Aw5g67p1HTpOp', '46frPLWZuQlMLyelCLYpL0', '6Q3K9gVUZRMZqZKrXovbM2', '4pDE8vhtcEb90Eq2SvdWyB', '4cktbXiXOapiLBMprHFErI', '4RVtBlHFKj51Ipvpfv5ER4', '4w5Grfxx5Rpup6mo5v6bW0', '55SbTToiQ8JTe524EeIn8b', '2anII3P3z1liigla0dKqrb', '12aAoKT43mVGdW1ofZVCwr', '76GbkhlVfUZDYK4VbVjX46', '5kWcIgj5CszKQBB0Lg7sGY', '4l0Mvzj72xxOpRrp6h8nHi', '4Q6wZlZhrwVVmGhJtmGUtG'])

In [ ]:
def get_song_date(song_id_list):
  buckets = math.ceil(len(song_id_list)/50)
  songs = np.array_split(song_id_list, buckets)
  song_date = list()
  for song_list in songs:
    song_list =  ','.join(song_list)
    url = f'https://api.spotify.com/v1/tracks?ids={song_list}'
    response = requests.get(url,
      headers = {
          'Authorization': f'Bearer {AUTH_TOKEN}'
      }        
    )
    json_response = response.json()
    for track in json_response['tracks']:
      song_date.append(track['album']['release_date'])

  return song_date

# get_song_popularity(['0mzvDQW1IgLuXAwD5YMN6i', '1L4f4kyGeaLdZeVhPLaD8j', '4IKSehYf4e9tjijDSZHQF2', '5vK8ILrHjXjdZJHp0AtEVJ', '7tAsrC5tNcbccUmMnRsmZI', '25mYG9U2wb2UnQrOVkSIbE', '1R9dlH1uHHuJJF8gp9WN19', '3uPrQDkNYwbOBwX5SHIXb4', '06Uhw7KLSBqAJqYy2fdjiN', '3FiX402udtjX777EHZUAbU', '7ayopKtCQTOEDpRB5xpU6I', '6oNF4mTDylJfuQF9Wqb3Hw', '1Z0GfmDY4xTdRNOCGWckyV', '2SwDECpUAk8jtZ31aXYQTe', '0jniFtpAyWN1hCtQdTllSg', '7rvKogufniHwoxQIgdcGde', '06JbxvU7hdbBw1eoEOUWW6', '2W48o4UcesMLJZMwoL6B3o', '2fDaX8wgmwzSYmdLmTDAIw', '1eKhGmIxCVwXHjxrh3aqT5', '7nByHsiIV08TNL6hG4nBBs', '0vC73pf42tqNW2ecQrdNtm', '36KSajo0NyH3eGgbS4AuJ7', '40ooZvE9wvUQ8oNfSzs73N', '4RIi1gNmKDzSH04Vvws2DK', '5uOdakRKQbRu6HJnorQ5on', '4hMKS6gf79eVQ70AkAXyCE', '2qSmlhY4zy0sZaGeKowauQ', '2vMRmWkG5H3Bi93ufKmWNl', '59Gf13MS5B8IyLG0ZjpWFX', '0zd0kuNdqeLgxuFm5SLYxH', '2VA9AQAyYgItqq1KE3AtKr', '555uSzxhjD72TQBEoPOfYT', '4Wdin6LVBOduIVlCKi8FYK', '4VD1tBHGujKPhkaEji4v5Q', '3JvKfv6T31zO0ini8iNItO', '3TArda30KGDCgfB3U9fmrC', '6gpTColYGFZ1q0isxHoE0u', '7Kn7TYpJT0fjv5he98acyq', '1H3uUBhA5X9YNdk1Ciw8ky', '3qDRfkAQLT4yQPtzYh9nrA', '2tKSIeksPzMglcIJVc83ID', '3bc2L7xATgHEXtNJdEJOv9', '4wwz9Ft24R5WOgWpZSOIo5', '1mFh7ZsIQvy10e3hGPoXQA', '1JoZpWoOpDZWCbfhVgjRiC', '7siCyYmarG5TfLQbWlNKJ7', '1s8QRwM3pW5ekIUKwlHsxo', '6OdcsvgPkIFmiyqDPgCaDy', '4jEEloo2pedhkrP5NBTl85', '2cJCPpy5Iwgon0wORjxJan', '2odlHl0cZ7Q1npc6RL07Ar', '7ifjN5DxB3r1hKnqjRDO1a', '53y2hHY9gPhOZzoEh7umgq', '14d79QOV2pvJHH9vlUfZwJ', '4OkXQF0aJwj3eECnph73GC', '4OxSe2OdRDAEx8BO3CA5uD', '3Dvams9yiJchXGj1OlOCil', '66tgeHKe9FfKCQnrfkFNxy', '4qu63nuBpdn0qHUHuObEj1', '6hYidRx2EkQ39bmZHvqqaI', '5sNLvOP38aQxpjUeJ2VQsP', '0wzK2m89t22oOayHYfV75U', '7s61Kg0qZhhFj5MQW7DDIB', '1D3irR3juCW9kOHfW1LXws', '11BAVEGi1ivJ6JWLqKUNrZ', '6vRCKJDtnrJEcyyzIMa0w0', '0JOHC5pkFmkZSLkwiumjTt', '5b19IPegprOMgLPDtC1Cyf', '7vEQ7hcDYobr5p4OkejYGK', '7k0wVBW3cvV6Xmb87JikIf', '1rEHH6Wrm0iuv61O1oM2nc', '1HNkqx9Ahdgi1Ixy2xkKkL', '7795WJLVKJoAyVoOtCWqXN', '3Vjmj8370BnHk6b3viNxfU', '0v1g5U1OAb0XBjKyA8R36i', '5Ohxk2dO5COHF1krpoPigN', '75JFxkI2RXiU7L9VXzMkle', '4PdLup4OaCGktZG8cvbkBz', '0fD6vPYWty2Jy4VVozWzfp', '5RoHs6F9G5eH1kl488diEu', '09wrW4mdaoBrOFY6KSR94h', '2WtPwpBC8qCRtzHUDZgwBN', '4m4dmiW49WkRkSWliJ5Zoj', '6sbNTk4raoJtHh92BUbLoi', '7ce20yLkzuXXLUhzIDoZih', '7GXG38022Aw5g67p1HTpOp', '46frPLWZuQlMLyelCLYpL0', '6Q3K9gVUZRMZqZKrXovbM2', '4pDE8vhtcEb90Eq2SvdWyB', '4cktbXiXOapiLBMprHFErI', '4RVtBlHFKj51Ipvpfv5ER4', '4w5Grfxx5Rpup6mo5v6bW0', '55SbTToiQ8JTe524EeIn8b', '2anII3P3z1liigla0dKqrb', '12aAoKT43mVGdW1ofZVCwr', '76GbkhlVfUZDYK4VbVjX46', '5kWcIgj5CszKQBB0Lg7sGY', '4l0Mvzj72xxOpRrp6h8nHi', '4Q6wZlZhrwVVmGhJtmGUtG'])

## Test

### TOPS

paises :
España, Argentina, Italia, Estados Unidos, Francia, Reino Unidos, Alemania, Brasil, Colombia, México, Marruecos, Japón, Chile, Los Paises Bajos, Perú, Paraguay, Bélgica, Portugal, Ucrania, Corea del Sur, República Dominicana, Irlanda, Suiza, Australia, Noruega, China, Canadá, India, Polonia, Rumanía, Suecia, Dinamarca, Uruguay, Rusia, Ecuador, Turquía, Grecia, Austria, Finlandia, Filipinas, Bolivia, Arabia Saudí, Bulgaria, Honduras, Tailandia, Egipto, Panamá, Israel, Costa Rica, Camboya, Lituania, Hungría, Nicaragua, Indonesia, Islandia, Áfirca del Sur, El Salvador, Vietnam, Surinam, República Checa, Guatemala, Taiwan, Pakistán, Eslovaquia, Luxemburgo, Nueva Zelanda, Estonia, Emirates Árabes Unidos, Hong Kong 

In [ ]:
playlists_ids_tops = {}

user_id = 'spotify'
url = f'https://api.spotify.com/v1/users/{user_id}/playlists?limit=50'
json_response = connect_to_endpoint(url, headers)
(json.dumps(json_response,  ensure_ascii=False,indent=2, sort_keys=True))
playlists_ids_tops = check_id_playlists(json_response, playlists_ids_tops)

next_token = json_response['next']

while next_token!= None:
    json_response = connect_to_endpoint(next_token, headers)
    (json.dumps(json_response,  ensure_ascii=False,indent=2, sort_keys=True))
    playlists_ids_tops = check_id_playlists(json_response, playlists_ids_tops)
    
    try:
        next_token = json_response['next']
    except:
        next_token = None

In [ ]:
print(len(playlists_ids_tops))
playlists_ids_tops #debería haber más pero solo salen estas, son 31. Faltan por añadir las de los moods

31


{'Global': '37i9dQZEVXbMDoHDwVN2tF',
 'Taiwán': '37i9dQZEVXbMnZEatlMSiu',
 'Turquía': '37i9dQZEVXbIVYVBNw9D5K',
 'Reino Unido': '37i9dQZEVXbLnolsZ8PSNw',
 'Dinamarca': '37i9dQZEVXbL3J0k32lWnN',
 'España': '37i9dQZEVXbNFJfN1Vw8d9',
 'Finlandia': '37i9dQZEVXbMxcczTSoGwZ',
 'Francia': '37i9dQZEVXbIPWwFssbupI',
 'Los Países Bajos': '37i9dQZEVXbKCF6dqVpDkS',
 'Noruega': '37i9dQZEVXbJvfa0Yxg7E7',
 'Suecia': '37i9dQZEVXbLoATJ81JYXz',
 'Estados Unidos': '37i9dQZEVXbLRQDuF5jeBp',
 'Austria': '37i9dQZEVXbKNHh6NIXu36',
 'Suiza': '37i9dQZEVXbJiyhoAPEfMK',
 'Bélgica': '37i9dQZEVXbJNSeeHswcKB',
 'Alemania': '37i9dQZEVXbJiZcmkrIHGU',
 'Nueva Zelanda': '37i9dQZEVXbM8SIrkERIYl',
 'Australia': '37i9dQZEVXbJPcfkRz0wJ0',
 'Irlanda': '37i9dQZEVXbKM896FDX8L1',
 'Luxemburgo': '37i9dQZEVXbKGcyg6TFGx6',
 'Italia': '37i9dQZEVXbIQnj7RRhdSX',
 'Polonia': '37i9dQZEVXbN6itCcaL3Tt',
 'Portugal': '37i9dQZEVXbKyJS56d1pgi',
 'Hong Kong': '37i9dQZEVXbLwpL8TjsxOG',
 'Malasia': '37i9dQZEVXbJlfUljuZExa',
 'Singapur': '37i9

In [ ]:
## CHECK WHICH COUNTRIES ARE AND HOW MANY ARE LEFT (TO FIND MANUALLY)

countries = 'España, Argentina, Italia, Estados Unidos, Francia, Reino Unidos, Alemania, Brasil, Colombia, México, Marruecos, Japón, Chile, Los Países Bajos, Perú, Paraguay, Bélgica, Portugal, Ucrania, Corea del Sur, República Dominicana, Irlanda, Suiza, Australia, Noruega, Canadá, India, Polonia, Rumanía, Suecia, Dinamarca, Uruguay, Rusia, Ecuador, Turquía, Grecia, Austria, Finlandia, Filipinas, Bolivia, Arabia Saudí, Bulgaria, Honduras, Tailandia, Egipto, Panamá, Israel, Costa Rica, Lituania, Hungría, Nicaragua, Indonesia, Islandia, Sudáfrica, El Salvador, Vietnam, Surinam, La República Checa, Guatemala, Taiwán, Eslovaquia, Luxemburgo, Nueva Zelanda, Estonia, EAU, Hong Kong'
countries_list = countries.split(', ')
print('Total (of our list):', len(countries_list))


done_countries = []
other_countries = []
left_countries = countries_list.copy()

for p in playlists_ids_tops.keys():
  name_country =p

  if name_country in countries_list:
    #the country found is in our list
    left_countries.remove(name_country)
  done_countries.append(name_country)


print('Done;', len(done_countries)) #31
print('Left:', len(left_countries)) #45

left_countries

Total (of our list): 66
Done; 31
Left: 40


['Argentina',
 'Reino Unidos',
 'Brasil',
 'Colombia',
 'Marruecos',
 'Japón',
 'Chile',
 'Perú',
 'Paraguay',
 'Ucrania',
 'Corea del Sur',
 'República Dominicana',
 'Canadá',
 'India',
 'Rumanía',
 'Uruguay',
 'Rusia',
 'Ecuador',
 'Grecia',
 'Filipinas',
 'Bolivia',
 'Arabia Saudí',
 'Bulgaria',
 'Honduras',
 'Tailandia',
 'Egipto',
 'Panamá',
 'Israel',
 'Costa Rica',
 'Hungría',
 'Nicaragua',
 'Indonesia',
 'Sudáfrica',
 'El Salvador',
 'Vietnam',
 'Surinam',
 'La República Checa',
 'Guatemala',
 'Eslovaquia',
 'EAU']

In [ ]:
manually_tops = ['https://open.spotify.com/playlist/37i9dQZEVXbMMy2roB9myp?si=351a34fa3a1f4f95', 'https://open.spotify.com/playlist/37i9dQZEVXbLnolsZ8PSNw?si=b660db803d1948dd', 'https://open.spotify.com/playlist/37i9dQZEVXbMXbN3EUUhlg?si=f5e7253c449f41f5',
                 'https://open.spotify.com/playlist/37i9dQZEVXbOa2lmxNORXQ?si=f2a6b9fc26544eb3', 'https://open.spotify.com/playlist/37i9dQZEVXbJU9eQpX8gPT?si=4d45a7aa878246ef', 'https://open.spotify.com/playlist/37i9dQZEVXbKXQ4mDTEBXq?si=c5d9aa4a08b3498a', 
                 'https://open.spotify.com/playlist/37i9dQZEVXbL0GavIqMTeb?si=672783ae485849fd', 'https://open.spotify.com/playlist/37i9dQZEVXbJfdy5b0KP7W?si=4364335667304201', 'https://open.spotify.com/playlist/37i9dQZEVXbNOUPGj7tW6T?si=54822ec7d3f2415a', 
                 'https://open.spotify.com/playlist/37i9dQZEVXbKkidEfWYRuD?si=01fe64f34be64239', 'https://open.spotify.com/playlist/37i9dQZEVXbNxXF4SkHj9F?si=c16aa76ed1d9466f', 'https://open.spotify.com/playlist/37i9dQZEVXbKAbrMR8uuf7?si=23074c660c714945',
                 'https://open.spotify.com/playlist/37i9dQZEVXbKj23U1GF4IR?si=dd3aec00bb7d454a', 'https://open.spotify.com/playlist/37i9dQZEVXbLZ52XmnySJg?si=f0a5fbfc458d4222', 'https://open.spotify.com/playlist/37i9dQZEVXbNZbJ6TZelCq?si=701f75cbb739447c', 
                 'https://open.spotify.com/playlist/37i9dQZEVXbMJJi3wgRbAy?si=be09226401094462', 'https://open.spotify.com/playlist/1IMY26ZqjwICaPzZGa1Lwf?si=7ae2a1655e274941', 'https://open.spotify.com/playlist/37i9dQZEVXbJlM6nvL1nD1?si=601b734b758a4626',
                 'https://open.spotify.com/playlist/37i9dQZEVXbJqdarpmTJDL?si=799b879c163640ba', 'https://open.spotify.com/playlist/37i9dQZEVXbNBz9cRCSFkY?si=dd115bf4e1a34f69', 'https://open.spotify.com/playlist/37i9dQZEVXbJqfMFK4d691?si=887998cf9f0d4216', 
                 'https://open.spotify.com/playlist/37i9dQZEVXbLrQBcXqUtaC?si=868ef72ed0834387', 'https://open.spotify.com/playlist/37i9dQZEVXbNfM2w2mq1B8?si=75245ed3ed2c423e', 'https://open.spotify.com/playlist/37i9dQZEVXbJp9wcIM9Eo5?si=33108dcfb72c4ff6', 
                 'https://open.spotify.com/playlist/37i9dQZEVXbMnz8KIWsvf9?si=dc8ed9324e8d466e', 'https://open.spotify.com/playlist/37i9dQZEVXbLn7RQmT5Xv2?si=855b510f836a4fce', 'https://open.spotify.com/playlist/37i9dQZEVXbKypXHVwk1f0?si=b4928ac2c5184a07', 
                 'https://open.spotify.com/playlist/37i9dQZEVXbJ6IpvItkve3?si=524fde2f34234828', 'https://open.spotify.com/playlist/37i9dQZEVXbMZAjGMynsQX?si=77292c6501bf4298', 'https://open.spotify.com/playlist/37i9dQZEVXbNHwMxAkvmF8?si=eb8b690f8b4c4e59', 
                 'https://open.spotify.com/playlist/37i9dQZEVXbISk8kxnzfCq?si=7aa3bf1b99394711', 'https://open.spotify.com/playlist/37i9dQZEVXbObFQZ3JLcXt?si=63eaf33671fa4a8e', 'https://open.spotify.com/playlist/37i9dQZEVXbMH2jvi6jvjk?si=23d27af40ed44f6f', 
                 'https://open.spotify.com/playlist/37i9dQZEVXbLxoIml4MYkT?si=8f94838e5b7e4a8e', 'https://open.spotify.com/playlist/37i9dQZEVXbLdGSmz6xilI?si=823dcff5aaa04a25', 'https://open.spotify.com/playlist/6zFJ2HTXC3d38AgpfcPGph?si=84e2ecb21bd24f59',
                 'https://open.spotify.com/playlist/37i9dQZEVXbIP3c3fqVrJY?si=6f024298ccdf4d40', 'https://open.spotify.com/playlist/37i9dQZEVXbLy5tBFyQvd4?si=2072564287c54801', 'https://open.spotify.com/playlist/37i9dQZEVXbKIVTPX9a2Sb?si=9f8e672f453e4f8e', 
                 'https://open.spotify.com/playlist/37i9dQZEVXbM4UZuIrvHvA?si=6a7709fc566d4f80']
print(len(manually_tops))              

40


In [ ]:
for i in manually_tops:
  playlist_id = i.split('/')[-1]
  playlist_id = playlist_id.split('?')[0]

  url = f'https://api.spotify.com/v1/playlists/{playlist_id}?limit=50'
  json_response = connect_to_endpoint(url, headers)
  (json.dumps(json_response,  ensure_ascii=False,indent=2, sort_keys=True))
  try:
    n = json_response['name'].split('Top\xa050: ')[1]
  except:
    n = json_response['name'].split('- ')[1]
  playlists_ids_tops[n] = playlist_id

In [ ]:
print(len(playlists_ids_tops))
playlists_ids_tops

70


{'Global': '37i9dQZEVXbMDoHDwVN2tF',
 'Taiwán': '37i9dQZEVXbMnZEatlMSiu',
 'Turquía': '37i9dQZEVXbIVYVBNw9D5K',
 'Reino Unido': '37i9dQZEVXbLnolsZ8PSNw',
 'Dinamarca': '37i9dQZEVXbL3J0k32lWnN',
 'España': '37i9dQZEVXbNFJfN1Vw8d9',
 'Finlandia': '37i9dQZEVXbMxcczTSoGwZ',
 'Francia': '37i9dQZEVXbIPWwFssbupI',
 'Los Países Bajos': '37i9dQZEVXbKCF6dqVpDkS',
 'Noruega': '37i9dQZEVXbJvfa0Yxg7E7',
 'Suecia': '37i9dQZEVXbLoATJ81JYXz',
 'Estados Unidos': '37i9dQZEVXbLRQDuF5jeBp',
 'Austria': '37i9dQZEVXbKNHh6NIXu36',
 'Suiza': '37i9dQZEVXbJiyhoAPEfMK',
 'Bélgica': '37i9dQZEVXbJNSeeHswcKB',
 'Alemania': '37i9dQZEVXbJiZcmkrIHGU',
 'Nueva Zelanda': '37i9dQZEVXbM8SIrkERIYl',
 'Australia': '37i9dQZEVXbJPcfkRz0wJ0',
 'Irlanda': '37i9dQZEVXbKM896FDX8L1',
 'Luxemburgo': '37i9dQZEVXbKGcyg6TFGx6',
 'Italia': '37i9dQZEVXbIQnj7RRhdSX',
 'Polonia': '37i9dQZEVXbN6itCcaL3Tt',
 'Portugal': '37i9dQZEVXbKyJS56d1pgi',
 'Hong Kong': '37i9dQZEVXbLwpL8TjsxOG',
 'Malasia': '37i9dQZEVXbJlfUljuZExa',
 'Singapur': '37i9

### MOOD

In [ ]:
playlists_ids_mood = {}

playlist_ids = ['https://open.spotify.com/playlist/37i9dQZF1DWV1PBrIG2weG?si=7bb3d347c75c4c0a', 'https://open.spotify.com/playlist/37i9dQZF1DX8CTD7HWYPDn?si=a335ed59da2f4880',
                'https://open.spotify.com/playlist/37i9dQZF1DXcjpPPxCzYRE?si=900dc809314c401f', 'https://open.spotify.com/playlist/37i9dQZF1DX9Dh2wgiAwVX?si=6cbc88c0d68f4b04', 'https://open.spotify.com/playlist/37i9dQZF1DXdPec7aLTmlC?si=291c0e98cd6b4cb6',
                'https://open.spotify.com/playlist/37i9dQZF1DXcZK031Zeh47?si=e15a2af567554bb5', 'https://open.spotify.com/playlist/37i9dQZF1DXdZjf8WgcTKM?si=881153f834774502', 'https://open.spotify.com/playlist/37i9dQZF1DWZqdNJSufucb?si=041d48677c004685',
                'https://open.spotify.com/playlist/37i9dQZF1DX9XIFQuFvzM4?si=42fe56c742354e8a', 'https://open.spotify.com/playlist/37i9dQZF1DWSf2RDTDayIx?si=307a5b120aa84619', 'https://open.spotify.com/playlist/37i9dQZF1DX7fvgalTu2lg?si=4556db502a624c81', 
                'https://open.spotify.com/playlist/37i9dQZF1DX3rxVfibe1L0?si=1350bdced62f4767', 'https://open.spotify.com/playlist/37i9dQZF1DX6VdMW310YC7?si=ff8816a299304244', 'https://open.spotify.com/playlist/37i9dQZF1DWUAZoWydCivZ?si=2f9bd36f78f04b76', 
                'https://open.spotify.com/playlist/37i9dQZF1DWX83CujKHHOn?si=ae9cbd280a224b1a', 'https://open.spotify.com/playlist/37i9dQZF1DX3YSRoSdA634?si=3880ba852b664a4e', 'https://open.spotify.com/playlist/37i9dQZF1DWYBO1MoTDhZI?si=50184219df4545a7', 
                'https://open.spotify.com/playlist/37i9dQZF1DX843Qf4lrFtZ?si=81527d249c0141cd', 'https://open.spotify.com/playlist/37i9dQZF1DX35X4JNyBWtb?si=ca65267f835b4dd0', 'https://open.spotify.com/playlist/37i9dQZF1DX6xZZEgC9Ubl?si=baeba9e080b14885',
                'https://open.spotify.com/playlist/37i9dQZF1DWZKuerrwoAGz?si=31cb9025b6a94594', 'https://open.spotify.com/playlist/37i9dQZF1DX1PiujNwN5Yy?si=b99cb92e03cb48b1', 'https://open.spotify.com/playlist/37i9dQZF1DWSqBruwoIXkA?si=dcc3fadce3d34d3a',
                'https://open.spotify.com/playlist/37i9dQZF1DX4OzrY981I1W?si=2fc059b5885f4f56', 'https://open.spotify.com/playlist/37i9dQZF1DX3R7OWWGN4gH?si=f5d87f1705eb40cf', 'https://open.spotify.com/playlist/37i9dQZF1DX59NCqCqJtoH?si=c052eb9afd5c4167', 
                'https://open.spotify.com/playlist/37i9dQZF1DWXLeA8Omikj7?si=587a46bda2db4e35', 'https://open.spotify.com/playlist/37i9dQZF1DX6mvEU1S6INL?si=42614550d3414137', 'https://open.spotify.com/playlist/37i9dQZF1DWVtFO27AQ6dF?si=15302c0b180a4e89', 
                'https://open.spotify.com/playlist/37i9dQZF1DWWTdxbiocWOL?si=58ce5563eaff4c05', 'https://open.spotify.com/playlist/37i9dQZF1DWZVAVMhIe3pV?si=3fa3418a0aa04cbc', 'https://open.spotify.com/playlist/37i9dQZF1DX2pSTOxoPbx9',
                'https://open.spotify.com/playlist/37i9dQZF1DWTtTyjgd08yp?si=e528fe5b066b4a9c', 'https://open.spotify.com/playlist/37i9dQZF1DX5hol82XuK24?si=4cd335b6307d46b8', 'https://open.spotify.com/playlist/37i9dQZF1DX4JTjLTaIQbE',
                'https://open.spotify.com/playlist/22wbHwDJ1l4otMV0oejUQ8', 'https://open.spotify.com/playlist/37i9dQZF1DX73EtbU4jEcn', 'https://open.spotify.com/playlist/5cfBRKU3wnQifn8Ruyfc7g',
                'https://open.spotify.com/playlist/37i9dQZF1DWVciwe52Zt0R', 'https://open.spotify.com/playlist/6sbVIgDGekGivQrEv6v7AC', 'https://open.spotify.com/playlist/37i9dQZF1DWYp5sAHdz27Y',
                'https://open.spotify.com/playlist/0vvXsWCC9xrXsKd4FyS8kM', 'https://open.spotify.com/playlist/37i9dQZF1DWXb9I5xoXLjp', 'https://open.spotify.com/playlist/37i9dQZF1DX3tuWZaHjp5y',
                'https://open.spotify.com/playlist/0p9bQ4Fc1NjyLGOXputNMs', 'https://open.spotify.com/playlist/37i9dQZF1DX4s3V2rTswzO', 'https://open.spotify.com/playlist/48bCki9uafyHh2xAD6Clby',
                'https://open.spotify.com/playlist/37i9dQZF1DX7Qo2zphj7u3', 'https://open.spotify.com/playlist/7JabddFr3Q6JPsND4v9Swf', 'https://open.spotify.com/playlist/37i9dQZF1EIdXWhf3FgUPh',
                'https://open.spotify.com/playlist/6IKQrtMc4c00YzONcUt7QH', 'https://open.spotify.com/playlist/0okKcRyYEwq8guFxzAPtlB', 'https://open.spotify.com/playlist/6c1dnhXrzdsTyZaQsVTy4K',
                'https://open.spotify.com/playlist/5MTTBDvnBW0Mg7vbKahCUG', 'https://open.spotify.com/playlist/7BYeFU22TFqGgAUQfGPQPk']

for i in playlist_ids:
  playlist_id = i.split('/')[-1]
  playlist_id = playlist_id.split('?')[0]

  url = f'https://api.spotify.com/v1/playlists/{playlist_id}'
  json_response = connect_to_endpoint(url, headers)
  (json.dumps(json_response,  ensure_ascii=False,indent=2, sort_keys=True))
  playlists_ids_mood[json_response['name']] = playlist_id



In [ ]:
len(playlist_ids)
len(playlists_ids_mood)

55

### PRUEBAS PARA GENIUS

In [ ]:
client_id = "Lj9Oez1zvNqTVDvbC_l5ym8U9KTguS5HVGNM05NnbtG0aMUcKcfXsMhbA49-0HMa"
client_access_token = "kUGHW6i2sdrMbIbkoI-KwldBJVyOjSvpJJJpK8_9Q1V56zFo_jl2iszUmeJ1sur8"

In [ ]:
import requests

def request_song_info(song_title, artist_name):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + client_access_token}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    return response

In [ ]:
request_song_info('Punto G', 'Quevedo')

In [ ]:
song_name = 'Punto G'
artist_name = 'Quevedo'
response = request_song_info(song_name, artist_name)
json = response.json()
remote_song_info = None

for hit in json['response']['hits']:
    if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
        remote_song_info = hit
        break

In [ ]:
print(remote_song_info)

API GENIUS

In [ ]:
genius_search_url = f"http://api.genius.com/search?q=Quevedo&access_token={client_access_token}"  # canciones de quevedo
response = requests.get(genius_search_url)
json_data = response.json()

lyrics_url = json_data['response']['hits'][0]['result']['url']  # url for Quevedo's first stored song's lyrics
print(lyrics_url)
response_lyrics = requests.get(lyrics_url)
response_lyrics


In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:
# set options to be headless
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("window-size=1200x600")

# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)

In [ ]:
url = "https://genius.com/Bizarrap-and-quevedo-quevedo-bzrp-music-sessions-vol-52-lyrics"
driver.get(url)

In [ ]:
results = driver.find_element(By.XPATH, "//*[@id='lyrics-root']/div[2]")
print(results.text)
# //*[@id="lyrics-root"]/div[2]

NoSuchElementException: ignored

BEAUTIFUL SOUP

In [ ]:
page_url = "https://genius.com/Bizarrap-and-quevedo-quevedo-bzrp-music-sessions-vol-52-lyrics"
page = requests.get(page_url)
html = BeautifulSoup(page.text, "html.parser")
#remove script tags that they put in the middle of the lyrics
[h.extract() for h in html('script')]
html
#at least Genius is nice and has a tag called 'lyrics'!
#lyrics = html.find('div', class_='Lyrics__Container-sc-1ynbvzw-6 YYrds').get_text() #updated css where the lyrics are based in HTML
#lyrics

In [ ]:
pl_esp = create_playlist_dict(playlists_ids_tops['Alemania'])

In [ ]:
import re

c = re.compile(r"Lyrics_Container")

page = requests.get("https://genius.com/Quevedo-punto-g-lyrics")
soup = BeautifulSoup(page.content, "html.parser")
lyrics = soup.find_all(id="lyrics-root-pin-spacer")

print(lyrics)


[]


In [ ]:
#function to scrape lyrics from genius
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics

In [ ]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics
# DEMO
print(scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics'))

In [ ]:
print(scrape_lyrics('Quevedo', 'punto g'))
# ReferentFragmentdesktop__Highlight-sc-110r0d9-1 jAzSMw

None


In [ ]:
for key,value in pl_esp.items():
  print('Song:',value['name'])
  print('Artists:',value['artists'])

Song: Punto G
Artists: [{'name': 'Quevedo'}]
Song: APA
Artists: [{'name': 'Mora'}, {'name': 'Quevedo'}]
Song: Hey Mor
Artists: [{'name': 'Ozuna'}, {'name': 'Feid'}]
Song: La Bachata
Artists: [{'name': 'Manuel Turizo'}]
Song: Quevedo: Bzrp Music Sessions, Vol. 52
Artists: [{'name': 'Bizarrap'}, {'name': 'Quevedo'}]
Song: Vista Al Mar
Artists: [{'name': 'Quevedo'}]
Song: CALENTÓN
Artists: [{'name': 'Mora'}]
Song: DOMINGO DE BOTE
Artists: [{'name': 'Mora'}]
Song: Besos Moja2
Artists: [{'name': 'Wisin & Yandel'}, {'name': 'ROSALÍA'}]
Song: Monotonía
Artists: [{'name': 'Shakira'}, {'name': 'Ozuna'}]
Song: PUNTO 40
Artists: [{'name': 'Rauw Alejandro'}, {'name': 'Baby Rasta'}]
Song: DESPECHÁ
Artists: [{'name': 'ROSALÍA'}]
Song: A Veces (feat. Feid)
Artists: [{'name': 'Paulo Londra'}, {'name': 'Feid'}]
Song: No Me digas Nada
Artists: [{'name': 'Quevedo'}]
Song: Feliz Cumpleaños Ferxxo
Artists: [{'name': 'Feid'}]
Song: Sin Señal
Artists: [{'name': 'Quevedo'}, {'name': 'Ovy On The Drums'}]
Song:

In [ ]:
for i in range(len(pl_esp)):
  print()

In [ ]:
name = ''

# Creating the Database

###Comments:

> Only keep some of the features (ej: not id, url, track_href...)
> Extra row in final csv to indicate country 

In [ ]:
#@title Test on a single playlist

# song_ids = retrieve_songs(playlists_ids_tops['España'])
# audio_feat = get_audio_features(song_ids)['audio_features']
# song_feats = []
# for song in audio_feat:
#   for del_ in ['type', 'uri', 'track_href', 'analysis_url']:
#     song.pop(del_)
#   song_feats.append(song)

  
# df = pd.DataFrame(song_feats)
# df = df.set_index('id')
# df['country'] = 'Spain'

In [ ]:
#On whole dataset --> Luxemburgo has only 44 songs in the playlist
df_top = pd.DataFrame()
for country in playlists_ids_tops:
  song_ids = retrieve_songs(playlists_ids_tops[country])
  audio_feat = get_audio_features(song_ids)['audio_features']
  song_feats = []
  for song in audio_feat:
    for del_ in ['type', 'uri', 'track_href', 'analysis_url']:
      try:
        song.pop(del_)
      except:
        pass
    song_feats.append(song)

  try: 
    tmp = pd.DataFrame(song_feats)
    tmp['country'] = country
    tmp['popularity'] = get_song_popularity(song_ids)
    tmp['release_date'] = get_song_date(song_ids)
    df_top = pd.concat([df_top,tmp], ignore_index = True)

  except:
    pass

In [ ]:
df_top

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,country,popularity,release_date
0,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.2380,131.121,3nqQXoyQOWXiESFLlDF1hG,156943,4,Global,100,2022-09-22
1,0.336,0.627,7,-7.463,1,0.0384,0.16400,0.000000,0.0708,0.3500,150.273,0bYg9bo50gSsH3LtXe2SQn,241107,4,Global,91,1994-10-28
2,0.637,0.643,4,-6.571,1,0.0519,0.13000,0.000002,0.1420,0.5330,97.008,0V3wPSX9ygBnCm8psDIegu,200690,4,Global,97,2022-10-21
3,0.735,0.478,2,-12.472,1,0.0293,0.18900,0.000002,0.3550,0.9470,107.682,2FRnf9qhLbvw8fu4IBXx78,262960,4,Global,87,1984-11-29
4,0.835,0.679,7,-5.329,0,0.0364,0.58300,0.000002,0.2180,0.8500,124.980,5ww2BF9slyYgNOk37BlC4u,162638,4,Global,97,2022-05-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3436,0.514,0.730,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.3340,171.005,0VjIjW4GlUZAMYd2vXMi3b,200040,4,EAU,90,2020-03-20
3437,0.247,0.299,9,-6.083,1,0.0315,0.89900,0.000000,0.1310,0.1720,177.115,6sCvvleqKbeyOkQDieBYgp,196520,4,EAU,76,2022-11-11
3438,0.486,0.545,11,-7.924,1,0.0336,0.06520,0.004740,0.0642,0.0385,150.187,4FyesJzVpA39hbYvcseO2d,123891,4,EAU,89,2022-10-17
3439,0.963,0.905,7,-4.053,1,0.1170,0.65400,0.048100,0.1070,0.5940,123.998,7twsd0lwXyy0ARdsTyzHoJ,178530,4,EAU,77,2022-11-18


In [ ]:
df_top.to_csv('/content/drive/MyDrive/Colab Notebooks/WebAnalytics/Spotify/song_feat_top.csv')

In [ ]:
d= Counter(df['id'])
# dict(sorted(d.items(), key=lambda item: item[1]), reverse=True)

import operator
sorted(d.items(), key=operator.itemgetter(1), reverse=True)[:30]


## MOODS

In [ ]:
moods = ['sport', 'sad', 'happy', 'chill', 'party', 'disconnecting', 'dark', 'revive', 'romance']

# # PARA CREAR EL DICT CON EL NOMBRE DE LAS PLAYLITS
# change = {}
# for i in playlists_ids_mood.keys():
#   print(i + ':')


d = {'temazos chill': 'chill',
 '¡Esta me la sé!': 'happy',
 'Relax en casa': 'chill',
 'Hits Alegres': 'happy',
 'Happy Hits!': 'happy',
 'Fuera estrés': 'revive',
 'canciones tristes': 'sad',
 'sad 😔': 'sad',
 "Feelin' Good": 'happy',
 'Happy Beats': 'happy',
 '¡Arriba los ánimos!': 'revive',
 'Mood Booster': 'revive',
 'Chill Tracks': 'chill',
 'Positive Vibes': 'happy',
 'Alone Again': 'sad',
 'Life Sucks': 'sad',
 'Good Vibes': 'happy',
 'young, wild & free': 'happy',
 'Energy Booster: Dance': 'revive',
 'Happy Favorites': 'happy',
 'gris': 'dark',
 'sad hour': 'sad',
 'my life is a movie': 'disconnecting',
 'Villain Mode': 'dark',
 'idk.': 'sad',
 'Brain Food': 'disconnecting',
 'You & Me': 'romance',
 'Maldita Hamburguesa': 'sport',
 'Calm Before the Storm': 'sad',
 'Energy Booster: Rock': 'revive',
 'tear drop': 'sad',
 'Deep Dark Indie': 'dark',
 'Spooky': 'dark',
  'Gym Tonic': 'sport',
  'SPORT MIX 2023  🏋️🤾‍♀️🚴🏃🥇🏆': 'sport',
  'Disconnect': 'disconnecting',
  'Romantic Ballads': 'romance',
  'ROMANTICAS 2022 ❤️ (Canciones romanticas)':'romance',
  'Darkside Mix': 'dark',
  'Sport Motivation 2022': 'sport',
  'Top Hits Workout':'sport',
  'Pura energía':'sport',
  'Workout Hits':'sport',
  'Lofi hip hop - beats to relax/study to': 'disconnecting',
  'The Most Romantic Songs in the World': 'romance',
  'Romance Playlist':'romance',
  'Classical Romance':'romance',
  'Hits Románticos':'romance',
  'Relax & Chill 2022 🌴': 'chill',
  'Dark & Stormy': 'dark',
  'Darkside Mix': 'dark',
  'Chill Vibes 2022 🌙': 'chill',
  'Mood Booster 2022 🌞 Positive Vibes, Happy Music': 'revive',
  'Cheer Up Hits': 'revive',
  'Disconnect yourself': 'disconnecting',
  'Disconnected': 'disconnecting'
  }

In [ ]:
#@title Test on a single playlist

# song_ids = retrieve_songs(playlists_ids_mood['Maldita hamburguesa'])
# audio_feat = get_audio_features(song_ids)['audio_features']
# song_feats = []
# for song in audio_feat:
#   for del_ in ['type', 'uri', 'track_href', 'analysis_url']:
#     song.pop(del_)
#   song_feats.append(song)

  
# df = pd.DataFrame(song_feats)
# df = df.set_index('id')
# df['mood'] = 'sport'
# df


In [ ]:
#On whole dataset --> 3056 rows × 15 columns
df_moods = pd.DataFrame()
for playlist in playlists_ids_mood:
  song_ids = retrieve_songs(playlists_ids_mood[playlist])
  audio_feat = get_audio_features(song_ids)['audio_features']
  song_feats = []
  for song in audio_feat:
    for del_ in ['type', 'uri', 'track_href', 'analysis_url']:
      try:
        song.pop(del_)
      except:
        pass
    song_feats.append(song)

  try: 
    tmp = pd.DataFrame(song_feats)
    tmp['mood'] = d[playlist]
    tmp['popularity'] = get_song_popularity(song_ids)
    tmp['release_date'] = get_song_date(song_ids)
    df_moods = pd.concat([df_moods,tmp], ignore_index = True)
  except:
    pass

# df_moods.to_csv('song_feat_moods.csv')

In [ ]:
df_moods

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,mood,popularity,release_date
0,0.766,0.333,7,-13.775,0,0.0558,0.82400,0.001700,0.1590,0.450,95.951,0mzvDQW1IgLuXAwD5YMN6i,126985,4,chill,49,2021-03-12
1,0.757,0.631,0,-6.886,1,0.0302,0.00703,0.008650,0.0931,0.264,125.983,1L4f4kyGeaLdZeVhPLaD8j,185813,4,chill,47,2021-12-09
2,0.734,0.425,9,-8.585,1,0.1170,0.79000,0.000013,0.1070,0.361,80.898,4IKSehYf4e9tjijDSZHQF2,133333,4,chill,51,2022-02-04
3,0.439,0.482,0,-8.226,1,0.0946,0.58300,0.000000,0.1420,0.368,179.884,5vK8ILrHjXjdZJHp0AtEVJ,185554,4,chill,48,2022-03-04
4,0.627,0.426,6,-5.994,1,0.0291,0.88300,0.001780,0.1100,0.498,67.520,7tAsrC5tNcbccUmMnRsmZI,189732,4,chill,46,2021-12-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4475,0.854,0.766,9,-4.697,0,0.1410,0.02420,0.000000,0.0793,0.784,118.004,2S5LNtRVRPbXk01yRQ14sZ,224258,4,disconnecting,66,2015-04-07
4476,0.693,0.822,4,-5.441,0,0.0439,0.00616,0.000002,0.3150,0.763,126.035,3zrYNl1aMdFrQkcOjKVr5u,223800,4,disconnecting,59,2012-06-22
4477,0.747,0.739,11,-3.318,0,0.0733,0.04750,0.000000,0.2130,0.663,127.959,3bhvFZ2eJMuWakoDYZdaR5,205040,4,disconnecting,48,2011-08-30
4478,0.804,0.836,6,-6.635,0,0.0890,0.32000,0.000000,0.2130,0.850,120.046,7mS8RbJji2UZAaguRGsOCH,223773,4,disconnecting,60,2003-11-13


In [ ]:
df_moods.to_csv('/content/drive/MyDrive/Colab Notebooks/WebAnalytics/Spotify/song_feat_moods.csv')

In [ ]:
Counter(df_moods['mood'])

4481

In [ ]:
## NUEVA IDEA
## con graph: como se parecen los paises (aqui da igual que se repitan las canciones en el csv porque así se ve como se parecen)
## después creamos modelo con las canciones de los moods (train y validation)
## limpiamos el data de los tops (para quitar repes) y lo usamos como test, para identificar el mood de esas canciones y ver que mood predomina en cada pais


## HAY QUE BUSCAR MAS PLAYLISTS PARA MOOD